In [9]:
from datetime import datetime

class Bcolors:
    """A class with names for ANSI escape sequences defining colors and font styles on terminal.
       Source: https://stackoverflow.com/questions/287871/how-do-i-print-colored-text-to-the-terminal
    """
    HEADER =    '\033[95m'
    OKBLUE =    '\033[94m'
    OKCYAN =    '\033[96m'
    OKGREEN =   '\033[92m'
    WARNING =   '\033[93m'
    FAIL =      '\033[91m'
    ENDC =      '\033[0m'
    BOLD =      '\033[1m'
    UNDERLINE = '\033[4m'


class Astro:
    """A class used to manage astro signs.
    Class attribute 'symbols' is a dictionary that defines names for Unicodes representing arial signs.
    """

    symbols = {
        'Aries':       '\U00002648',
        'Taurus':      '\U00002649',
        'Gemini':      '\U0000264A',
        'Cancer':      '\U0000264B',
        'Leo':         '\U0000264C',
        'Virgo':       '\U0000264D',
        'Libra':       '\U0000264E',
        'Scorpius':    '\U0000264F',
        'Sagittarius': '\U00002650',
        'Capricorn':   '\U00002651',
        'Aquarius':    '\U00002652',
        'Pisces':      '\U00002653'
    }

    def sign_name(month, day):
        """Returns a string with astro sign name for given month and day."""
        if month == 1:    astro_sign = 'Capricorn'   if (day < 20) else 'Aquarius'
        elif month == 2:  astro_sign = 'Aquarius'    if (day < 19) else 'Pisces'
        elif month == 3:  astro_sign = 'Pisces'      if (day < 21) else 'Aries'
        elif month == 4:  astro_sign = 'Aries'       if (day < 20) else 'Taurus'
        elif month == 5:  astro_sign = 'Taurus'      if (day < 21) else 'Gemini'
        elif month == 6:  astro_sign = 'Gemini'      if (day < 21) else 'Cancer'
        elif month == 7:  astro_sign = 'Cancer'      if (day < 23) else 'Leo'
        elif month == 8:  astro_sign = 'Leo'         if (day < 23) else 'Virgo'
        elif month == 9:  astro_sign = 'Virgo'       if (day < 23) else 'Libra'
        elif month == 10: astro_sign = 'Libra'       if (day < 23) else 'Scorpius'
        elif month == 11: astro_sign = 'Scorpius'    if (day < 22) else 'Sagittarius'
        elif month == 12: astro_sign = 'Sagittarius' if (day < 22) else 'Capricorn'
        return astro_sign


class Pesel:
    """A class used to manage PESEL number.
    The PESEL validation algorithm is implemented on the basis of information from the website
    https://obywatel.gov.pl/pl/dokumenty-i-dane-osobowe/czym-jest-numer-pesel.
    """

    def __init__(self, pesel):
        """Initializes 'pesel' attribute of the created object with the value of the argument.
        If the argument is not correct, prints a warning
        and initializes 'pesel' attribute with an empty string.
        """
        self.set_pesel(pesel)

    def __str__(self):
        """Returns a string representing Pesel object. The string consists of
        PESEL number and data that is derived from it (birthday, gender, astro sign).
        In case of empty 'pesel' attribute, returns the string 'No PESEL value'.
        """
        if self.pesel == "":
            return Bcolors.BOLD + "No PESEL value." + Bcolors.ENDC
        birthday = f"{self.get_birthday().strftime('%Y-%m-%d')}"
        astro_symbol = Astro.symbols[Astro.sign_name(self.get_month(), self.get_day())]
        return Bcolors.BOLD + f"{self.pesel}, " + \
               Bcolors.ENDC + Bcolors.OKBLUE + \
               f"{birthday}, {self.get_gender()}  " + Bcolors.ENDC + \
               astro_symbol

    def set_pesel(self, pesel):
        """Sets the 'pesel' attribute of the created object with the value of the argument.
        If the argument is not correct, prints a warning
        and sets 'pesel' attribute with an empty string.
        """
        warn_text = Bcolors.FAIL + \
                    f"WARNING: class Pesel, method set_pesel(\"{pesel}\") - "
        if len(pesel) != 11 or not pesel.isdecimal():
            self.pesel = ""
            print(warn_text + "wrong characters or length." + Bcolors.ENDC)
            return
        self.pesel = pesel
        if self.calculate_control_sum() != int(pesel[10]):
            self.pesel = ""
            print(warn_text + "wrong control sum." + Bcolors.ENDC)
            return
        try:
            datetime(self.get_year(), self.get_month(), self.get_day())
        except:
            self.pesel = ""
            print(warn_text + "incorrect date." + Bcolors.ENDC)

    def get_year(self):
        """Gets the year of birth as int.
        If the 'pesel' attribute contains an empty string, the method returns None.
        """
        if self.pesel == "":
            return None
        begining = ["19", "20", "21", "22", "18"]
        return int(begining[int(self.pesel[2]) // 2] + self.pesel[0:2])

    def get_month(self):
        """Gets the month of birth as int.
        If the 'pesel' attribute contains an empty string, the method returns None.
        """
        if self.pesel == "":
            return None
        return int(self.pesel[2:4]) - (int(self.pesel[2]) // 2) * 20

    def get_day(self):
        """Gets the day of birth as int.
        If the 'pesel' attribute contains an empty string, the method returns None.
        """
        if self.pesel == "":
            return None
        return int(self.pesel[4:6])

    def get_birthday(self):
        """Gets the birthday as datatime.
        If the year, month, day do not form correct date, returns None.
        """
        try:
            return datetime(self.get_year(), self.get_month(), self.get_day())
        except:
            return None

    def get_gender(self):
        """Gets the gender: "F" for female, "M" for male.
        If the 'pesel' attribute contains an empty string, the method returns None.
        """
        if self.pesel == "":
            return None
        return "F" if self.pesel[9] in {'0', '2', '4', '6', '8'} else "M"

    def calculate_control_sum(self):
        """Calculates the control sum.
        If the 'pesel' attribute contains an empty string, the method returns None.
        """
        if self.pesel == "":
            return None
        ct = [1, 3, 7, 9, 1, 3, 7, 9, 1, 3]
        sum = 0
        for i in range(10):
            sum += (int(self.pesel[i]) * ct[i])
        return 10 - int(str(sum)[-1])

print("Classes: Bcolors, Astro, Pesel created.")

Classes: Bcolors, Astro, Pesel created.


In [10]:
class Person:
    """A class used to manage personal data.
    """
    def __init__(self, first_name, last_name, pesel):
        """Initializes 'first_name', 'last_name', and 'pesel' attributes
        of the created object with the values of the arguments.
        If the argument is not correct, prints a warning
        and initializes attribute with an empty string.
        """
        self.set_first_name(first_name)
        self.set_last_name(last_name)
        self.set_pesel(pesel)

    def __str__(self):
        """Returns a string representing Person object.
        """
        return "first_name: " + Bcolors.BOLD + f"{self.first_name}\n" + Bcolors.ENDC +\
               "last_name: " + Bcolors.BOLD + f"{self.last_name}\n" + Bcolors.ENDC +\
               f"pesel, birthday, gender: {self.pesel}\n"

    def set_first_name(self, first_name):
        """Sets the 'first_name' attribute of the created object with the value of the argument.
        If the argument is not correct, prints a warning
        and sets the attribute with an empty string.
        """
        self.first_name = first_name if Person.is_correct_name(first_name) else ""

    def set_last_name(self, last_name):
        """Sets the 'last_name' attribute of the created object with the value of the argument.
        If the argument is not correct, prints a warning
        and sets theattribute with an empty string.
        """
        self.last_name = last_name if Person.is_correct_name(last_name) else ""

    def set_pesel(self, pesel):
        """Sets the 'pesel' attribute of the created object with the value of the argument.
        If the argument is not correct, prints a warning
        and sets the attribute with an empty string.
        """
        self.pesel = Pesel(pesel)

    def get_first_name(self):
        """Gets the first name.
        If the 'first_name' attribute contains an empty string, the method returns None.
        """
        if self.first_name == "":
            return None
        return self.first_name

    def get_last_name(self):
        """Gets the last name.
        If the 'last_name' attribute contains an empty string, the method returns None.
        """
        if self.last_name == "":
            return None
        return self.last_name

    def get_pesel(self):
        """Gets pesel.
        If the 'pesel' attribute contains no pesel, the method returns None.
        """
        if self.pesel.get_year() == None:
            return None
        return self.pesel

    def is_correct_name(name):
        """
        Checks if the parameter represents name, that is if contains only letters
        and the first is uppercase.
        If yes, returns True. If not, prints a warning and returns False.
        """
        warn_text = Bcolors.FAIL + \
                    f"WARNING: class Person, method is_correct_name(\"{name}\") - "
        if not name.isalpha() or not name[0].isupper():
            print(warn_text + \
                  "non-letter or first character not uppercase" + Bcolors.ENDC)
            return False
        else:
            return True

print("Class Person created.")

Class Person created.


In [3]:
help(Person)

Help on class Person in module __main__:

class Person(builtins.object)
 |  Person(first_name, last_name, pesel)
 |  
 |  A class used to manage personal data.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, first_name, last_name, pesel)
 |      Initializes 'first_name', 'last_name', and 'pesel' attributes
 |      of the created object with the values of the arguments.
 |      If the argument is not correct, prints a warning
 |      and initializes attribute with an empty string.
 |  
 |  __str__(self)
 |      Returns a string representing Person object.
 |  
 |  get_first_name(self)
 |      Gets the first name.
 |      If the 'first_name' attribute contains an empty string, the method returns None.
 |  
 |  get_last_name(self)
 |      Gets the last name.
 |      If the 'last_name' attribute contains an empty string, the method returns None.
 |  
 |  get_pesel(self)
 |      Gets pesel.
 |      If the 'pesel' attribute contains no pesel, the method returns None.
 |  
 |  is_corre

In [11]:
pesels = ("96041369189", "02241916791", "03290941521", "99050283147", "98100144331", "98030692285", "01262733619", "00311262898", "98090516376", "97102351398", "?")
first_names = ("Bolesława", "Witold", "Subisława", "Klaudyna", "Aleks", "Delfina", "Samuel", "Otto", "Zygfryd", "Dobrogost", "?")
last_names =("Kubas", "Miotk", "Piętka", "Boratyńska", "Wojtkiewicz", "Moskała", "Sobiech", "Toporek", "Mrożek", "Skałecki", "?")


persons = []
for i in range(11):
    persons.append(Person(first_names[i], last_names[i], pesels[i]))

for p in persons:
    print(p)

for p in persons:
    print(p.get_first_name(), p.get_last_name(), p.get_pesel())


first_name: Bolesława
last_name: Kubas
pesel, birthday, gender: 96041369189, 1996-04-13, F  ♈

first_name: Witold
last_name: Miotk
pesel, birthday, gender: 02241916791, 2002-04-19, M  ♈

first_name: Subisława
last_name: Piętka
pesel, birthday, gender: 03290941521, 2003-09-09, F  ♍

first_name: Klaudyna
last_name: Boratyńska
pesel, birthday, gender: 99050283147, 1999-05-02, F  ♉

first_name: Aleks
last_name: Wojtkiewicz
pesel, birthday, gender: 98100144331, 1998-10-01, M  ♎

first_name: Delfina
last_name: Moskała
pesel, birthday, gender: 98030692285, 1998-03-06, F  ♓

first_name: Samuel
last_name: Sobiech
pesel, birthday, gender: 01262733619, 2001-06-27, M  ♋

first_name: Otto
last_name: Toporek
pesel, birthday, gender: 00311262898, 2000-11-12, M  ♏

first_name: Zygfryd
last_name: Mrożek
pesel, birthday, gender: 98090516376, 1998-09-05, M  ♍

first_name: Dobrogost
last_name: Skałecki
pesel, birthday, gender: 97102351398, 1997-10-23, M  ♏

first_name: 
last_name: 
pesel, birthday, gender

<br>

## Task 1

- Think of two derived classes of the `Person` class (e.g. Student, Lecturer).

- Think of at least one attribute that appears in one of the derived classes but does not appear in the other.

- Think of at least one method that appears in one of the derived classes but does not appear in the other.

There is nothing to prevent you from doing the above stage of work in a group of two or more people. It is worth discussing what the types of attributes are and what the parameters and values ​​returned by the methods are.

Implement both derived classes. Create attributes, provide getters and setters, implement sensible object listing. Implement the methods.

Remember to use `super()` wherever possible.

Test the operation of both classes.

In [12]:
# Solution of exercise 1.
class Student(Person):
  def __init__(self, first_name, last_name, pesel, year_of_admission):
    super().__init__(first_name, last_name, pesel)
    self.set_year_of_admission(year_of_admission)
  def set_year_of_admission(self, year_of_admission):
      self.year_of_admission = year_of_admission
  def get_year_of_admission(self):
    return self.year_of_admission
  def is_graduate(self):
    return "2020" in self.year_of_admission
  def __str__(self):
    graduate = "Yes" if self.is_graduate() else "No"
    return super().__str__() + f"Year of admission: {self.year_of_admission}, \nGraduate: {graduate}"


class Teacher(Person):
  def __init__(self, first_name, last_name, pesel, science_degree):
    super().__init__(first_name, last_name, pesel)
    self.set_science_degree(science_degree)
  def set_science_degree(self, science_degree):
    self.science_degree = science_degree
  def get_science_degree(self):
    return self.science_degree
  def has_online_teaching_permission(self):
    return "Doctor of Science" in self.science_degree
  def __str__(self):
    online_permission = "Yes" if self.has_online_teaching_permission() else "No"
    return super().__str__() + f"Science degree: {self.science_degree}, \nOnline teaching permission: {online_permission}"


pesels = ("96041369189", "02241916791", "03290941521", "99050283147", "98100144331", "98030692285", "01262733619", "00311262898", "02351262868", "97102351398")
first_names = ("Bolesława", "Witold", "Subisława", "Klaudyna", "Aleks", "Delfina", "Samuel", "Otto", "Zygfryd", "Dobrogost")
last_names =("Kubas", "Miotk", "Piętka", "Boratyńska", "Wojtkiewicz", "?", "Sobiech", "Toporek", "Mrożek", "Skałecki")
science_degrees =("Master", "Doctor of Science", "Doctor of Science","Master","Master", "Doctor of Science", "Doctor of Science","Master","Doctor of Science","Master")
year_of_admissions = ("2020", "2022", "2020", "2021", "2023", "2023", "2022", "2020", "2021", "2023")
persons = []
for i in range(len(pesels)):
    pesel = pesels[i]
    first_name = first_names[i]
    last_name = last_names[i]
    science_degree = science_degrees[i]
    year_of_admission = year_of_admissions[i]
    people = Teacher(first_name, last_name, pesel, science_degree)
    persons.append(people)
for people in persons:
    print(str(people))
    print()
print("-----------------------------------------------------------------------")
persons1 = []
for i in range(len(pesels)):
    pesel = pesels[i]
    first_name = first_names[i]
    last_name = last_names[i]
    science_degree = science_degrees[i]
    year_of_admission = year_of_admissions[i]
    people = Student(first_name, last_name, pesel, year_of_admission)
    persons1.append(people)
for people in persons1:
    print(str(people))
    print()

first_name: Bolesława
last_name: Kubas
pesel, birthday, gender: 96041369189, 1996-04-13, F  ♈
Science degree: Master, 
Online teaching permission: No

first_name: Witold
last_name: Miotk
pesel, birthday, gender: 02241916791, 2002-04-19, M  ♈
Science degree: Doctor of Science, 
Online teaching permission: Yes

first_name: Subisława
last_name: Piętka
pesel, birthday, gender: 03290941521, 2003-09-09, F  ♍
Science degree: Doctor of Science, 
Online teaching permission: Yes

first_name: Klaudyna
last_name: Boratyńska
pesel, birthday, gender: 99050283147, 1999-05-02, F  ♉
Science degree: Master, 
Online teaching permission: No

first_name: Aleks
last_name: Wojtkiewicz
pesel, birthday, gender: 98100144331, 1998-10-01, M  ♎
Science degree: Master, 
Online teaching permission: No

first_name: Delfina
last_name: 
pesel, birthday, gender: 98030692285, 1998-03-06, F  ♓
Science degree: Doctor of Science, 
Online teaching permission: Yes

first_name: Samuel
last_name: Sobiech
pesel, birthday, gender

<br>

## Task 2

Replace the `Person` class with an abstract class. Write code that proves that it is an abstract class.

Hint: how to do this is described in lecture 8.

In [6]:
# Solution of exercise 2.
from abc import ABC, abstractmethod
class Person(ABC):

  def __init__(self, first_name, last_name, pesel):
    self.set_first_name(first_name)
    self.set_last_name(last_name)
    self.set_pesel(pesel)

  @abstractmethod
  def __str__(self):
    return "first_name: " + Bcolors.BOLD + f"{self.first_name}\n" + Bcolors.ENDC +\
               "last_name: " + Bcolors.BOLD + f"{self.last_name}\n" + Bcolors.ENDC +\
               f"pesel, birthday, gender: {self.pesel}\n"

  @abstractmethod
  def set_first_name(self, first_name):
    self.first_name = first_name if Person.is_correct_name(first_name) else ""

  def set_last_name(self, last_name):
    self.last_name = last_name if Person.is_correct_name(last_name) else ""

  def set_pesel(self, pesel):
    self.pesel = Pesel(pesel)

  def get_first_name(self):
    if self.first_name == "":
        return None
    return self.first_name

  def get_last_name(self):
    if self.last_name == "":
        return None
    return self.last_name

  def get_pesel(self):
    if self.pesel.get_year() == None:
        return None
    return self.pesel

  def is_correct_name(name):
    warn_text = Bcolors.FAIL + \
                f"WARNING: class Person, method is_correct_name(\"{name}\") - "
    if not name.isalpha() or not name[0].isupper():
        print(warn_text + \
              "non-letter or first character not uppercase" + Bcolors.ENDC)
        return False
    else:
        return True
person = Person("Tom", "Born", "96041369189")
print("Person have been added")



TypeError: Can't instantiate abstract class Person with abstract methods __str__, set_first_name

<br>

## Task 3

Add a method to both derived classes that will have the same name but a slightly different action.

An example of such a method is given in lecture 8 for the `StoreOrder` and `OnlineOrder` classes - it is the `get_address` method.

Test this method.

In [13]:
# Solution of exercise 3.
class Student(Person):
  def __init__(self, first_name, last_name, pesel, year_of_admission):
    super().__init__(first_name, last_name, pesel)
    self.set_year_of_admission(year_of_admission)
  def set_year_of_admission(self, year_of_admission):
      self.year_of_admission = year_of_admission
  def get_year_of_admission(self):
    return self.year_of_admission
  def is_graduate(self):
    return "2020" in self.year_of_admission
  def __str__(self):
    graduate = "Yes" if self.is_graduate() else "No"
    return super().__str__() + f"Year of admission: {self.year_of_admission}, \nGraduate: {graduate}, \n{self.i_am()}"
  def i_am(self):
    return f"I am a student and I entered this university in {self.year_of_admission}"


class Teacher(Person):
  def __init__(self, first_name, last_name, pesel, science_degree):
    super().__init__(first_name, last_name, pesel)
    self.set_science_degree(science_degree)
  def set_science_degree(self, science_degree):
    self.science_degree = science_degree
  def get_science_degree(self):
    return self.science_degree
  def has_online_teaching_permission(self):
    return "Doctor of Science" in self.science_degree
  def __str__(self):
    online_permission = "Yes" if self.has_online_teaching_permission() else "No"
    return super().__str__() + f"Science degree: {self.science_degree}, \nOnline teaching permission: {online_permission}, \n{self.i_am()}"
  def i_am(self):
    return f"I'm a teacher and I'm a {self.science_degree}"



pesels = ("96041369189", "02241916791", "03290941521", "99050283147", "98100144331", "98030692285", "01262733619", "00311262898", "02351262868", "97102351398")
first_names = ("Bolesława", "Witold", "Subisława", "Klaudyna", "Aleks", "Delfina", "Samuel", "Otto", "Zygfryd", "Dobrogost")
last_names =("Kubas", "Miotk", "Piętka", "Boratyńska", "Wojtkiewicz", "?", "Sobiech", "Toporek", "Mrożek", "Skałecki")
science_degrees =("Master", "Doctor of Science", "Doctor of Science","Master","Master", "Doctor of Science", "Doctor of Science","Master","Doctor of Science","Master")
year_of_admissions = ("2020", "2022", "2020", "2021", "2023", "2023", "2022", "2020", "2021", "2023")
persons = []
for i in range(len(pesels)):
    pesel = pesels[i]
    first_name = first_names[i]
    last_name = last_names[i]
    science_degree = science_degrees[i]
    year_of_admission = year_of_admissions[i]
    people = Teacher(first_name, last_name, pesel, science_degree)
    persons.append(people)
for people in persons:
    print(str(people))
    print()
print("-----------------------------------------------------------------------")
persons1 = []
for i in range(len(pesels)):
    pesel = pesels[i]
    first_name = first_names[i]
    last_name = last_names[i]
    science_degree = science_degrees[i]
    year_of_admission = year_of_admissions[i]
    people = Student(first_name, last_name, pesel, year_of_admission)
    persons1.append(people)
for people in persons1:
    print(str(people))
    print()

first_name: Bolesława
last_name: Kubas
pesel, birthday, gender: 96041369189, 1996-04-13, F  ♈
Science degree: Master, 
Online teaching permission: No, 
I'm a teacher and I'm a Master

first_name: Witold
last_name: Miotk
pesel, birthday, gender: 02241916791, 2002-04-19, M  ♈
Science degree: Doctor of Science, 
Online teaching permission: Yes, 
I'm a teacher and I'm a Doctor of Science

first_name: Subisława
last_name: Piętka
pesel, birthday, gender: 03290941521, 2003-09-09, F  ♍
Science degree: Doctor of Science, 
Online teaching permission: Yes, 
I'm a teacher and I'm a Doctor of Science

first_name: Klaudyna
last_name: Boratyńska
pesel, birthday, gender: 99050283147, 1999-05-02, F  ♉
Science degree: Master, 
Online teaching permission: No, 
I'm a teacher and I'm a Master

first_name: Aleks
last_name: Wojtkiewicz
pesel, birthday, gender: 98100144331, 1998-10-01, M  ♎
Science degree: Master, 
Online teaching permission: No, 
I'm a teacher and I'm a Master

first_name: Delfina
last_name: